In [1]:
import auxiliary_tools
import pandas as pd
from geopy.distance import geodesic
import matplotlib.path as mplPath
import numpy as np
import pickle
from tqdm import tqdm
from math import sqrt
from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools
creator.create("min_fitness", base.Fitness, weights=(-1.0, -1.0))
creator.create("individual", list, fitness=creator.min_fitness)
import evolution
tqdm.pandas()

/usr/local/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def get_nearst_point(coordinate, list_virtual_points, distance):
    aux = geodesic(kilometers=distance).destination(coordinate, -180)
    left = [aux.latitude, aux.longitude]
    aux = geodesic(kilometers=distance).destination(coordinate, 0)
    right = [aux.latitude, aux.longitude]
    aux = geodesic(kilometers=distance).destination(coordinate, 45)
    c_u_left = [aux.latitude, aux.longitude]
    aux = geodesic(kilometers=distance).destination(coordinate, 135)
    c_u_right = [aux.latitude, aux.longitude]
    aux = geodesic(kilometers=distance).destination(coordinate, 225)
    c_d_left = [aux.latitude, aux.longitude]
    aux = geodesic(kilometers=distance).destination(coordinate, 315)
    c_d_right = [aux.latitude, aux.longitude]
    coordinate_polygon = [c_u_left, c_u_right, c_d_left, c_d_right, c_u_left]
    polygon_path = mplPath.Path(coordinate_polygon)
    
    count_in = 0
    nearest_virtual_stop = [np.nan, np.nan]
    virtual_stop_distance_min = 999999
    virtual_stop_duration_min = np.nan
    for virtual_stop in list_virtual_points:
        if polygon_path.contains_point(virtual_stop):
            count_in+=1
            virtual_stop_found_distance, virtual_stop_found_duration = auxiliary_tools.getGeoDistanceETA_OSRM(coordinate, virtual_stop, 5001, 'walking')
            if virtual_stop_found_distance < virtual_stop_distance_min:
                virtual_stop_distance_min = virtual_stop_found_distance
                virtual_stop_duration_min = virtual_stop_found_duration
                nearest_virtual_stop = virtual_stop
    
    nearest_virtual_stop = list(nearest_virtual_stop)
    nearest_virtual_stop.extend([virtual_stop_distance_min, virtual_stop_duration_min])
    return nearest_virtual_stop

def get_route_distance(individual, penalty_const):
    ori_dest = [(first, second) for first, second in zip(individual, individual[1:])]
    penalty_sum = 0
    for pair in ori_dest:
        if pair[0] != pair[1]:
            penalty_sum+=penalty_const
    o_fitness = []
    for od_r in ori_dest:
        o_fitness.append(auxiliary_tools.getGeoDistanceETA_OSRM(od_r[0], od_r[1], 5000, 'driving'))
        
    fitness_distance = np.sum([f[0] for f in o_fitness]) + penalty_sum
    fitness_duration = np.sum([f[1] for f in o_fitness]) + penalty_sum
    return fitness_distance, fitness_duration

In [3]:
def get_result_day(df_rides, day, method, d):
    df_rides = df_rides[df_rides.DAY == day]
    
    if method == 'CORNER-TO-CORNER':
        df_virtual_stops = pd.read_pickle("df_virtual_stops.pkl")
        df_virtual_stops.columns = map(str.upper, df_virtual_stops.columns)
        virtual_stops_list = list(zip(df_virtual_stops['LATITUDE'].tolist(), df_virtual_stops['LONGITUDE'].tolist()))
        
        df_rides[['VS_LATITUDE', 'VS_LONGITUDE', 'WALKING_DISTANCE', 'WALKING_DURATION']] = df_rides.progress_apply(lambda row: pd.Series(get_nearst_point([row['LATITUDE'], row['LONGITUDE']], virtual_stops_list, d)), axis=1)
    elif method == 'GA 0':
        data_ga = pickle.load(open("data_ga_evol_4.pkl", "rb"))
        [list_generations_mean_fitness, list_generations_std_fitness,
          list_generations_min_fitness, list_generations_max_fitness,
          list_population_generations, list_best_individuo_generations,
          list_best_individuo_fitness_generations] = data_ga
        
        virtual_stops_list = list_best_individuo_generations[-1]

        df_rides[['VS_LATITUDE', 'VS_LONGITUDE', 'WALKING_DISTANCE', 'WALKING_DURATION']] = df_rides.progress_apply(lambda row: pd.Series(get_nearst_point([row['LATITUDE'], row['LONGITUDE']], virtual_stops_list, d)), axis=1)
    elif method == 'GA 1':
        data_ga = pickle.load(open("data_ga_evol_5.pkl", "rb"))
        [list_generations_mean_fitness, list_generations_std_fitness,
          list_generations_min_fitness, list_generations_max_fitness,
          list_population_generations, list_best_individuo_generations,
          list_best_individuo_fitness_generations] = data_ga
        
        virtual_stops_list = list_best_individuo_generations[-1]

        df_rides[['VS_LATITUDE', 'VS_LONGITUDE', 'WALKING_DISTANCE', 'WALKING_DURATION']] = df_rides.progress_apply(lambda row: pd.Series(get_nearst_point([row['LATITUDE'], row['LONGITUDE']], virtual_stops_list, d)), axis=1)
    elif method == 'GA 2':
        
        data_ga = pickle.load(open("data_ga_evol_8.pkl", "rb"))
        
        [list_generations_mean_fitness, list_generations_std_fitness,
          list_generations_min_fitness, list_generations_max_fitness,
          list_population_generations, list_best_individuo_generations,
          list_best_individuo_fitness_generations] = data_ga
        
        virtual_stops_list = list_best_individuo_generations[-1]
        
        df_rides[['VS_LATITUDE', 'VS_LONGITUDE', 'WALKING_DISTANCE', 'WALKING_DURATION']] = df_rides.progress_apply(lambda row: pd.Series(get_nearst_point([row['LATITUDE'], row['LONGITUDE']], virtual_stops_list, d)), axis=1)
    elif method == 'DOOR-TO-DOOR':
        df_rides['VS_LATITUDE'] = df_rides['LATITUDE']
        df_rides['VS_LONGITUDE'] = df_rides['LONGITUDE']
        df_rides['WALKING_DISTANCE'] = 0
        df_rides['WALKING_DURATION'] = 0
    
    df_rides = df_rides.dropna()
    
    results_data = []
    for van_id in tqdm(set(df_rides.VAN_ID)):
        df_van = df_rides[df_rides.VAN_ID == van_id].sort_values(by='TIMESTAMP').reset_index(drop=True)
        locations_list = list(zip(df_van['VS_LATITUDE'].tolist(), df_van['VS_LONGITUDE'].tolist()))
        client_distance_cost = df_van.WALKING_DISTANCE.sum()
        client_duration_cost = df_van.WALKING_DURATION.sum()
        
        operator_distance_cost, operator_duration_cost = get_route_distance(locations_list, 0)
        
        results_data.append([van_id, day, method, client_distance_cost, client_duration_cost, operator_distance_cost, operator_duration_cost])
        
    df_result_ga = pd.DataFrame(results_data, columns=['VAN_ID', 'DAY', "METHOD", "CLIENT_DISTANCE_COST",
                                                       "CLIENT_DURATION_COST", "OPERATOR_DISTANCE_COST",
                                                       "OPERATOR_DURATION_COST"])
    df_result_ga.to_pickle("../RESULTS/DF_RESULT_"+day+"_"+method+".pkl")

In [4]:
df_rides, df_others = auxiliary_tools.load_dataset()
df_rides = df_rides.reset_index(drop=True)

# proposals:  123402
# requests 121427
# rides 46616


In [5]:
day_list = list(set(df_rides.DAY))
method_list = ['DOOR-TO-DOOR', 'CORNER-TO-CORNER', 'GA 0', 'GA 1', 'GA 2']

In [6]:
for day_str in day_list:
    for method_str in method_list:
        get_result_day(df_rides, day_str, method_str, 0.4)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.